In [1]:
import pandas as pd
import altair as alt
from urllib.error import HTTPError

In [2]:
def claim(claim_hash, cols_claim, data_claim):
    try:
        df_claim = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"],
        )
    except:
        return pd.DataFrame(data_claim[claim_hash],columns=cols_claim[claim_hash])
    if(len(df_claim.columns)==0):
        return pd.DataFrame(data_claim[claim_hash],columns=cols_claim[claim_hash])
    return df_claim

In [3]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [4]:
user_stats = '1'
airdrop_claims = '2'
lba_deposits = '3'
mars_roi = ''

In [5]:
cols_claim = {
    user_stats : ['SENDER', 'DURATION', 'AMOUNT'],
    airdrop_claims : ['SENDER','AMOUNT','TIME'],
    lba_deposits : ['SENDER','AMOUNT','DENOM','ACTION','TIME','AIRDROP'],
    mars_roi : ['3 months','6 months','9 months','12 months','15 months','18 months']
}

In [6]:
data_claim = {
               mars_roi : [
          [1.695441e+07, 1.329435e+07,  4.737131e+06,  3.118435e+06,  1.032555e+06,  4.229810e+07],
           [3.000000e+00,  6.000000e+00,  9.000000e+00,  1.200000e+01,  1.500000e+01,  1.800000e+01],
          [1.000000e+00,  2.800000e+00,  5.200000e+00,  8.000000e+00, 1.120000e+01,  1.470000e+01]],
    user_stats : [['user1_1',3,10],
                  ['user1_2',3,3],
                  ['user1_3',3,4],
                  ['user1_4',3,67],
                  ['user1_5',3,33],
                 ['user1',9,20],
                 ['user1',18,15],
                 ['user2',3,10],
                 ['user2',6,120],
                 ['user2',18,13],
                 ['user2',3,10],
                 ['user3',6,120],
                 ['user3',18,13]],
    airdrop_claims :
        [
            ['user1',300,'2021-09-21T07:00:00Z'],
            ['user2',70,'2021-09-21T07:00:00Z'],
            ['user3',34,'2021-09-21T07:00:00Z'],
            ['user1_1',132,'2021-09-21T07:00:00Z'],
            ['user4',132,'2021-09-21T07:00:00Z']
        ],
    lba_deposits : [['user1_1',20,'MARS','deposit','2021-09-21T08:00:00Z',True],
                    ['user1_1',50,'UST','deposit','2021-09-21T08:00:00Z',None],
                    ['user2',70,'MARS','deposit','2021-09-21T08:00:00Z',None],
                    ['user2',80,'UST','deposit','2021-09-21T08:00:00Z',None],
                    ['user2',10,'UST','withdraw','2021-09-21T09:00:00Z',None],
                    ['user1',70,'MARS','deposit','2021-09-21T08:00:00Z',True],
                    ['user4',132,'MARS','deposit','2021-09-21T09:00:00Z',None],
                    ['user1_5',132,'MARS','deposit','2021-09-21T09:00:00Z',None]]}

### Data

In [7]:
mars_roi_df = claim(mars_roi,cols_claim,data_claim)
mars_roi_df.columns = [c.lower() for c in mars_roi_df.columns]
mars_roi_df.index = ['UST deposited','n_months','boost']
mars_roi_df

,3 months,6 months,9 months,12 months,15 months,18 months
UST deposited,16954410.0,13294350.0,4737131.0,3118435.0,1032555.0,42298100.0
n_months,3.0,6.0,9.0,12.0,15.0,18.0
boost,1.0,2.8,5.2,8.0,11.2,14.7


**User stats**

In [8]:
user_stats_df = claim(user_stats,cols_claim,data_claim)
user_stats_df.columns = [c.lower() for c in user_stats_df.columns]
user_stats_df

,sender,duration,amount
0,user1_1,3,10
1,user1_2,3,3
2,user1_3,3,4
3,user1_4,3,67
4,user1_5,3,33
5,user1,9,20
6,user1,18,15
7,user2,3,10
8,user2,6,120
9,user2,18,13


**Airdrop**

In [9]:
airdrop_claims_df = claim(airdrop_claims,cols_claim,data_claim)
airdrop_claims_df.columns = [c.lower() for c in airdrop_claims_df.columns]
airdrop_claims_df

,sender,amount,time
0,user1,300,2021-09-21T07:00:00Z
1,user2,70,2021-09-21T07:00:00Z
2,user3,34,2021-09-21T07:00:00Z
3,user1_1,132,2021-09-21T07:00:00Z
4,user4,132,2021-09-21T07:00:00Z


**LBA deposits**

In [10]:
lba_deposits_df = claim(lba_deposits,cols_claim,data_claim)
lba_deposits_df.columns = [c.lower() for c in lba_deposits_df.columns]
lba_deposits_df['time'] = pd.to_datetime(lba_deposits_df.time)
lba_deposits_df['amount'] = lba_deposits_df.apply(lambda row: -row.amount if row.action=='withdraw' else row.amount, axis=1)
lba_deposits_df

,sender,amount,denom,action,time,airdrop
0,user1_1,20,MARS,deposit,2021-09-21 08:00:00+00:00,True
1,user1_1,50,UST,deposit,2021-09-21 08:00:00+00:00,None
2,user2,70,MARS,deposit,2021-09-21 08:00:00+00:00,None
3,user2,80,UST,deposit,2021-09-21 08:00:00+00:00,None
4,user2,-10,UST,withdraw,2021-09-21 09:00:00+00:00,None
5,user1,70,MARS,deposit,2021-09-21 08:00:00+00:00,True
6,user4,132,MARS,deposit,2021-09-21 09:00:00+00:00,None
7,user1_5,132,MARS,deposit,2021-09-21 09:00:00+00:00,None


**Hourly LBA**

In [11]:
lba_deposits_df['hour'] = lba_deposits_df.time.dt.strftime("%Y-%m-%d %H:00")

In [12]:
mars = lba_deposits_df[lba_deposits_df.denom=='MARS']
ust = lba_deposits_df[lba_deposits_df.denom=='UST']

In [22]:
df_mars = mars.groupby(['denom','hour']).amount.sum().reset_index().sort_values(by='hour')
df_mars['cumsum'] = df_mars.amount.cumsum()
df_ust = ust.groupby(['denom','hour']).amount.sum().reset_index().sort_values(by='hour')
df_ust['cumsum'] = df_ust.amount.cumsum()
lba_deposits_hourly_df = df_ust.append(df_mars)
lba_deposits_hourly_df

,denom,hour,amount,cumsum
0,UST,2021-09-21 08:00,130,130
1,UST,2021-09-21 09:00,-10,120
0,MARS,2021-09-21 08:00,160,160
1,MARS,2021-09-21 09:00,264,424


In [25]:
m = lba_deposits_hourly_df[lba_deposits_hourly_df.denom=='MARS']
u = lba_deposits_hourly_df[lba_deposits_hourly_df.denom=='UST']
d = m.merge(u, on='hour',how='outer').fillna(0)
d['cumsum'] = d.cumsum_x / d.cumsum_y
d['denom'] = 'MARS Price'
d = d[['hour','cumsum','denom']]
lba_deposits_hourly_df = lba_deposits_hourly_df.append(d).fillna(0)

**User Deposits**

In [39]:
user_mars = mars.groupby('sender').amount.sum().rename('mars').reset_index().set_index('sender')
user_ust = ust.groupby('sender').amount.sum().rename('ust').reset_index().set_index('sender')

In [40]:
users_deposits = user_mars.join(user_ust, how='outer').fillna(0)
users_deposits['total'] = users_deposits.mars+users_deposits.ust
users_deposits['mars_price'] = users_deposits.ust/users_deposits.mars

In [41]:
top_dep_by_total = users_deposits.sort_values(by='total', ascending=False).head(5)
top_dep_by_total

,mars,ust,total,mars_price
sender,,,,
user2,70,70.0,140.0,1.0
user1_5,132,0.0,132.0,0.0
user4,132,0.0,132.0,0.0
user1,70,0.0,70.0,0.0
user1_1,20,50.0,70.0,2.5


In [42]:
top_dep_by_mars_price = users_deposits.sort_values(by='mars_price', ascending=False).head(5)
top_dep_by_mars_price

,mars,ust,total,mars_price
sender,,,,
user1_1,20,50.0,70.0,2.5
user2,70,70.0,140.0,1.0
user1,70,0.0,70.0,0.0
user1_5,132,0.0,132.0,0.0
user4,132,0.0,132.0,0.0


**Users partecipation**

In [43]:
aidrop_users = airdrop_claims_df.sender.unique()

In [44]:
aidrop_users_df = pd.DataFrame(aidrop_users, columns=['sender'])
aidrop_users_df['type'] = 'Airdrop'

In [45]:
p1_users = user_stats_df.sender.unique()
p1_users_df = pd.DataFrame(p1_users, columns=['sender'])
p1_users_df['type'] = 'Phase1'

In [46]:
users_type = aidrop_users_df.append(p1_users_df)
users_type = users_type.merge(users_type.groupby('sender').type.count().rename('n_types').reset_index(), on=['sender'], how='left')
users_type['new_type'] = users_type.apply(lambda row: row.type if row.n_types==1 else 'Airdrop/Phase1', axis=1)
users_type = users_type[["sender","new_type"]].drop_duplicates()
users_type

,sender,new_type
0,user1,Airdrop/Phase1
1,user2,Airdrop/Phase1
2,user3,Airdrop/Phase1
3,user1_1,Airdrop/Phase1
4,user4,Airdrop
6,user1_2,Phase1
7,user1_3,Phase1
8,user1_4,Phase1
9,user1_5,Phase1


**LBA Mars origin**

In [47]:
lba_deposits_df['origin'] = lba_deposits_df.airdrop.apply(lambda x: 'Airdrop' if x else 'Phase 1')
lba_deposits_df['origin'] = lba_deposits_df.airdrop.apply(lambda x: 'Airdrop' if x else 'Phase 1')
mars_source = lba_deposits_df[lba_deposits_df.denom=='MARS'].groupby('origin').amount.sum()
mars_source = mars_source.reset_index()
mars_source

,origin,amount
0,Airdrop,90
1,Phase 1,334


**Metrics**

In [48]:
mars_source = users_type.merge(lba_deposits_df[lba_deposits_df.denom=='MARS'], on='sender').groupby('new_type').amount.sum()
mars_source = mars_source.reset_index()
mars_source

,new_type,amount
0,Airdrop,132
1,Airdrop/Phase1,160
2,Phase1,132


In [49]:
users_aidrop_eligible = 66103
perc_airdrop_eligible = len(airdrop_claims_df.sender.unique())/users_aidrop_eligible
perc_airdrop_eligible

7.56395322451326e-05

In [50]:
#Phase 1 + airdrop
mars_total = 60000000
mars = lba_deposits_hourly_df[lba_deposits_hourly_df.denom=='MARS']
act_mars_lba = mars[mars.hour == mars.hour.max()]["cumsum"].values[0]
perc_mars_in_lba = act_mars_lba/mars_total
perc_mars_in_lba

7.066666666666667e-06

In [51]:
usts = lba_deposits_hourly_df[lba_deposits_hourly_df.denom=='UST']
act_usts_lba = usts[usts.hour == mars.hour.max()]["cumsum"].values[0]
act_price = act_mars_lba/act_usts_lba
act_price

3.533333333333333

In [52]:
users_p1 = len(user_stats_df.sender.unique())
users_p1_lba = len(set(user_stats_df.sender.unique()).intersection(set(lba_deposits_df.sender.unique())))
perc_p1_lba = users_p1_lba/users_p1
perc_p1_lba

0.5

**P1 MARS rewards**

In [53]:
boost = pd.DataFrame([1,2.8,5.2,8,11.2,14.7],columns=['boost'],index=[3,6,9,12,15,18])

In [54]:
boost = user_stats_df.groupby('duration').sum().join(boost)
boost['amount_boosted'] = boost.amount * boost.boost

In [55]:
boost['amount_boosted_mars'] = (boost['amount_boosted']/boost['amount_boosted'].sum())*50000000

In [56]:
boost['amount_per_ust'] = boost['amount_boosted_mars']/boost.amount
boost

,amount,boost,amount_boosted,amount_boosted_mars,amount_per_ust
duration,,,,,
3,137,1.0,137.0,4.519364e+06,32988.058323
6,240,2.8,672.0,2.216798e+07,92366.563304
9,20,5.2,104.0,3.430758e+06,171537.903279
18,41,14.7,602.7,1.988190e+07,484924.457346


In [57]:
user_stats_df = user_stats_df.merge(boost.reset_index()[['duration','amount_per_ust']], on=['duration'])
user_stats_df['mars'] = user_stats_df.amount * user_stats_df.amount_per_ust
user_stats_df

,sender,duration,amount,amount_per_ust,mars
0,user1_1,3,10,32988.058323,3.298806e+05
1,user1_2,3,3,32988.058323,9.896417e+04
2,user1_3,3,4,32988.058323,1.319522e+05
3,user1_4,3,67,32988.058323,2.210200e+06
4,user1_5,3,33,32988.058323,1.088606e+06
5,user2,3,10,32988.058323,3.298806e+05
6,user2,3,10,32988.058323,3.298806e+05
7,user1,9,20,171537.903279,3.430758e+06
8,user1,18,15,484924.457346,7.273867e+06
9,user2,18,13,484924.457346,6.304018e+06


**% il LBA from P1 for each user**

In [58]:
user_p1_mars = user_stats_df.groupby('sender').mars.sum()

In [59]:
user_p1_perc_mars = lba_deposits_df[lba_deposits_df.denom=='MARS'].groupby('sender').sum().join(user_p1_mars).fillna(0)
user_p1_perc_mars['perc_p1_mars_lba'] = user_p1_perc_mars.apply(lambda row: 0 if ((row.mars == 0) or (row.amount==0)) else row.amount / row.mars,axis=1)
user_p1_perc_mars

,amount,mars,perc_p1_mars_lba
sender,,,
user1,70,1.070462e+07,0.000007
user1_1,20,3.298806e+05,0.000061
user1_5,132,1.088606e+06,0.000121
user2,70,1.804777e+07,0.000004
user4,132,0.000000e+00,0.000000


**APR P2**

In [60]:
tot_mars_rewards = 5000000
tot_ust_rewards = 5000000
price = 0.2
amount_ust_input = 10
amount_mars_input = 10

In [61]:
ust_apr = tot_ust_rewards*price/act_usts_lba
ust_apr

8333.333333333334

In [62]:
mars_apr = tot_mars_rewards/act_mars_lba
mars_apr

11792.452830188678

In [63]:
df = pd.DataFrame([[mars_apr,ust_apr],['MARS','UST']]).T
df.columns = ['ROI','Token']
roi_phase_2 = df

In [64]:
ust_rewards_input = tot_ust_rewards/(act_usts_lba+amount_ust_input)*amount_ust_input
ust_rewards_input

384615.3846153846

In [65]:
mars_rewards_input = tot_mars_rewards/(act_mars_lba+amount_mars_input)*amount_mars_input
mars_rewards_input

115207.3732718894

**What have users deposited**

In [66]:
df = lba_deposits_df.groupby(['sender','denom']).amount.sum().reset_index()
df2 = df[df.amount>0].groupby('sender').denom.count().rename('n_denom')
df3 = df.merge(df2.reset_index(), on='sender')
df3['dep_type'] = df3.apply(lambda row: row.denom if row.n_denom==1 else 'MARS & UST', axis=1)

In [67]:
user_dep_type = df3.groupby(['dep_type']).sender.count().reset_index()
user_dep_type

,dep_type,sender
0,MARS,3
1,MARS & UST,4


In [68]:
def get_max_domain_date(df, time_field, n_hours):
    if((pd.Timestamp(df[time_field].max()) - 
                pd.Timestamp(df[time_field].min())).total_seconds()/3600 < n_hours):
        max_date = (pd.Timestamp(df[time_field].min()) + pd.to_timedelta(n_hours, unit='h')).strftime("%Y-%m-%dT%H:%M:%SZ")
    else:
        max_date = df[time_field].max()
    return max_date

### Charts

In [69]:
roi_phase_2['% ROI'] = roi_phase_2.ROI.apply(lambda x: '% ' + str(round(x,2)))

In [70]:
roi_phase_2_chart = alt.Chart(roi_phase_2).mark_bar().encode(
            y=alt.Y("ROI"),
            x=alt.X('Token', sort="ascending",axis=alt.Axis(labelAngle=0)),
            tooltip=['Token:Q',"ROI:Q"],
            color=alt.Color('Token:O',
                        scale=alt.Scale(scheme='lightorange'),
                        legend=None),
        )
text = roi_phase_2_chart.mark_text(
                align='center',
                baseline='middle',
                dy=-15,  # Nudges text to right so it doesn't appear on top of the bar
                fontSize=25
            ).encode(
                text='% ROI:N'
            )

(roi_phase_2_chart + text).properties(width=300).configure_view(strokeOpacity=0)
         

alt.LayerChart(...)

In [71]:
user_p1_perc_mars.columns = ['MARS locked in Phase 2','MARS obtained from Phase 1','Percentage of MARS deposited from Phase 1']
user_p1_perc_mars_chart = alt.Chart((user_p1_perc_mars)).mark_bar().encode(
    x=alt.X('Percentage of MARS deposited from Phase 1', sort="ascending", bin=True),
    y="count()",
    tooltip=['Percentage of MARS deposited from Phase 1','count()'],
    color=alt.Color(scale=alt.Scale(scheme='redpurple'),legend=None),
).configure_mark(color='#f58766').properties(height=300).configure_view(strokeOpacity=0)
user_p1_perc_mars_chart

alt.Chart(...)

In [88]:
user_p1_perc_mars['fake'] = 'Phase1 & Phase 2 participants'

In [89]:
alt.Chart(user_p1_perc_mars).mark_boxplot(extent='min-max').encode(
                        y=alt.X(field="fake", axis=alt.Axis(labelAngle=-90, title='')),
                        x='Percentage of MARS deposited from Phase 1:Q'
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [72]:
user_dep_type

,dep_type,sender
0,MARS,3
1,MARS & UST,4


In [73]:
user_dep_type.columns = ['What users have deposited','Number of Users']
user_dep_type_chart = alt.Chart(user_dep_type).mark_arc(innerRadius=60).encode(
    theta=alt.Theta(field="Number of Users", type="quantitative"),
    color=alt.Color(field="What users have deposited", type="nominal",
            sort=['MARS & UST','MARS','UST'],
            scale=alt.Scale(scheme='lightorange'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-10,
            direction='vertical')),
    tooltip=['What users have deposited','Number of Users']
).configure_view(strokeOpacity=0)
user_dep_type_chart

alt.Chart(...)

In [74]:
lba_deposits_hourly_df

,denom,hour,amount,cumsum
0,UST,2021-09-21 08:00,130,130
1,UST,2021-09-21 09:00,-10,120
0,MARS,2021-09-21 08:00,160,160
1,MARS,2021-09-21 09:00,264,424


In [75]:
domain = ['MARS','UST']
range_ = ['#f9bf94','#a72327']
lba_deposits_hourly_df.columns = ['Token','Time','Amount_','Amount']
max_date = get_max_domain_date(lba_deposits_hourly_df,'Time',10)
lba_deposits_hourly_df_chart = alt.Chart(lba_deposits_hourly_df).mark_line(point = True).encode(
    x=alt.X('Time:T',scale=alt.Scale(domain=(lba_deposits_hourly_df.Time.min(),max_date))),
    y=alt.X('Amount:Q',scale=alt.Scale(domain=(0,lba_deposits_hourly_df['Amount'].max()+10))),
    color=alt.Color('Token:N', 
                sort=domain,
                scale=alt.Scale(domain=domain, range=range_),
                legend=alt.Legend(
                            orient='none',
                            padding=5,
                            legendY=0,
                            direction='horizontal')),
    tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d %H:%M'),'UST deposited:Q','Lockup period:N']
).properties(height=400).configure_view(strokeOpacity=0)
lba_deposits_hourly_df_chart

alt.Chart(...)

In [76]:
mars_source

,new_type,amount
0,Airdrop,132
1,Airdrop/Phase1,160
2,Phase1,132


In [77]:
mars_source.columns = ['User Participation','Amount of MARS locked in Phase 2']
mars_source_chart = alt.Chart(mars_source).mark_arc(innerRadius=60).encode(
    theta=alt.Theta(field="Amount of MARS locked in Phase 2", type="quantitative"),
    color=alt.Color(field="User Participation", type="nominal",
            sort=['Airdrop/Phase1','Phase1','Airdrop'],
            scale=alt.Scale(scheme='lightorange'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-10,
            direction='vertical')),
    tooltip=['User Participation','Amount of MARS locked in Phase 2']
).configure_view(strokeOpacity=0)
mars_source_chart

alt.Chart(...)